In [191]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import pandas as pd

In [192]:
def do_scrape():
    link = 'https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX%3A32017R0745&from=IT&fbclid=IwAR0RGdpi82vgk1kXnPpnOybyEB61Gow6R39Fx97y91xEOaUq6rOCL0R3goA#d1e4765-1-1'

    response = requests.get(link)
    #write(response.text) 
    return response.content

In [193]:
def write(response):
    # Write html file from data scrapped
    with open(f"scrap.html","w",encoding="utf-8")as html_file:
        html_file.write(response.text)    

In [194]:
def get_info():
    soup = BeautifulSoup(do_scrape(), 'html')

    # Loop till we hit first chapter
    looper = soup.find('p', class_='ti-section-1').find_next_sibling()
    #looper = looper.find_previous_sibling()

    all_chapters = {}
    all_articles = {}
    all_info = []
    chapter = 0
    article = 0
    print('CHAPTER I')
    while True:
        # Remove (Scope and definitions)
        if looper in soup.find_all('p', class_='ti-section-2'):
            looper = looper.find_next_sibling()
        # Remove (Subject of matter and scope)
        if looper in soup.find_all('p', class_='sti-art'):
            looper = looper.find_next_sibling()
            
        # Store chapters
        if looper in soup.find_all('p', class_='ti-section-1'):
            print(looper.text.replace(' ', '_'))
            all_chapters[chapter] = all_articles
            all_articles = {}
            chapter += 1
            if looper.text == 'CHAPTER X':
                break
            
        # Store the next Article
        if looper in soup.find_all('p', class_='ti-art'):
            print(looper.text.replace('\u00A0', '_'))
            all_articles[article] = all_info
            article += 1
            all_info = []
            
        # Store tekst 
        if looper not in soup.find_all('p', class_='ti-art'):
            all_info.append(looper.text)
        looper = looper.find_next_sibling()
    return all_chapters